In [ ]:
! pip install langchain langchain_community langchain_qdrant langchain_groq markdown unstructured FlagEmbedding nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 67.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.

In [ ]:
from langchain_core.embeddings import Embeddings
from pydantic import BaseModel, Field
from typing import Any, Dict

from langchain_qdrant.sparse_embeddings import SparseVector

from FlagEmbedding import BGEM3FlagModel

import nltk
nltk.download('punkt')


class CommonEmbedding(BaseModel, Embeddings):

    client: Any = None
    encode_kwargs: Dict[str, Any] = Field(default_factory=dict)
    """Keyword arguments to pass when calling the `encode` method of the model."""

    def __init__(self,
                 model:BGEM3FlagModel,
                 **kwargs):

        super().__init__(**kwargs)
        self.client = model

    def embed_documents(self, texts):

        embeddings = self.client.encode(texts,
                                        **self.encode_kwargs)

        if self.encode_kwargs.get('return_sparse', False):
            return [SparseVector(indices=i.keys(),
                                 values = i.values()) for i in embeddings['lexical_weights']]

        if self.encode_kwargs.get('return_dense', False):
            return embeddings['dense_vecs']

    def embed_query(self, text):
        return self.embed_documents([text])[0]


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in BGEM3Embegging has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in BGEM3Embegging has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
bgem3 = BGEM3FlagModel('BAAI/bge-m3', device='cuda', normalize_embeddings=True)
bgem3_sparse = CommonEmbedding(bgem3,
                               encode_kwargs={'return_sparse':True,
                                              'return_dense':False,
                                              'batch_size':1})
bgem3_dense = CommonEmbedding(bgem3,
                               encode_kwargs={'return_sparse':False,
                                              'return_dense':True,
                                              'batch_size':1})


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(array([ 0.012215,  0.05396 , -0.04675 , ..., -0.00853 ,  0.01257 ,
         0.02397 ], dtype=float16),
 SparseVector(indices=[104, 78987, 15263, 170, 434], values=[0.06005859375, 0.18408203125, 0.1761474609375, 0.145263671875, 0.155517578125]))

In [ ]:
from langchain.document_loaders import DirectoryLoader #, MarkdownLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader


# Path to your Obsidian markdown notes
directory_path = "../../data/lesha_obsi_sample"

# Initialize the DirectoryLoader to load .md files
loader = DirectoryLoader(
    directory_path,
    glob="**/*.md",
    loader_cls=UnstructuredMarkdownLoader,
    loader_kwargs={"encoding": "utf-8"}
)

docs = loader.load()
print(len(docs))
print(docs[0])

65
page_content='title: "Генеративный подход к классификации" source: "https://education.yandex.ru/handbook/ml/article/generativnyj-podhod-k-klassifikacii" author: published: created: 2024-10-22 description: "Как использовать распределение меток классов в задаче классификации. LDA, QDA и наивный байес" tags: - "clippings"

Как использовать распределение меток классов в задаче классификации. LDA, QDA и наивный байес

Классификационные модели, которые мы рассматривали в предыдущих параграфах, нацелены непосредственно на оценку $P(Y \vert X)$. Такие модели называются дискриминативными.

К ним относится, например, логистическая регрессия: она предлагает оценку $\hat P(y=1 \vert x) = \sigma(w\^Tx)$. В процессе обучения дискриминативные модели подбирают разделяющую поверхность (гиперплоскость в случае логистической регрессии). Новые объекты дискриминативная модель классифицирует в зависимости от того, по какую сторону от разделяющей поверхности они лежат.

Например, обучившись на изображения

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# tbd: switch to markdown splitter to split by .md headers
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_qdrant.qdrant import RetrievalMode

qvs = QdrantVectorStore.from_documents(splits,
                                        embedding=bgem3_dense,
                                        sparse_embedding=bgem3_sparse,
                                        location=':memory:',
                                        collection_name='temp',
                                        retrieval_mode=RetrievalMode.HYBRID)

In [ ]:
qvs.search('смещение и разброс', 'similarity')

[Document(metadata={'source': '/content/data/12.1. Bias-variance decomposition.md', '_id': 'be068f5a88da4b83b687a513bcbc5bc8', '_collection_name': 'temp'}, page_content='$$ \\mathbb{V}\\text{ar}_X[a(x, X)] = \\mathbb{E}_X \\left[ a(x, X) - \\mathbb{E}_X[a(x, X)] \\right]^2 $$\n\n— дисперсия (разброс) предсказаний алгоритма в зависимости от обучающей выборки $X$;\n\n$$ \\sigma^2 = \\mathbb{E}x \\mathbb{E}\\varepsilon[y(x, \\varepsilon) - f(x)]^2 $$\n\n— неустранимый шум в данных.\n\nСмещение показывает, насколько хорошо с помощью данного алгоритма можно приблизить истинную зависимость $f$, а разброс характеризует чувствительность алгоритма к изменениям в обучающей выборке. Например, деревья маленькой глубины будут в большинстве случаев иметь высокое смещение и низкий разброс предсказаний, так как они не могут слишком хорошо запомнить обучающую выборку. А глубокие деревья, наоборот, могут безошибочно выучить обучающую выборку и потому будут иметь высокий разброс в зависимости от выборки,

In [ ]:
retriever = qvs.as_retriever(search_type = 'similarity',
                             search_kwargs={'k': 5})

In [ ]:
from langchain_groq.chat_models import ChatGroq
import os

os.environ['GROQ_API_KEY'] = '*'
llm = ChatGroq(model='gemma2-9b-it')

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
print(f"prompt: {prompt}")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


prompt: input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("опиши Метод Дэвида-Скина?", verbose=True)

'Метод Дэвида-Скина (Dawid, Skene, 1979) одновременно находит значения качества исполнителей и ответы на вопросы, согласующиеся с наблюдаемыми данными в наибольшей степени.  \nОн использует вероятности, чтобы учесть ошибки разметчиков и найти наиболее вероятные ответы. \nМетод широко используется в краудсорсинге для агрегации данных. \n\n\n'